<a href="https://colab.research.google.com/github/qghrjf646/Quishing_detection_AI/blob/main/QR_code_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Loading PNG images from a folders into tensors

#benign_folder = "/content/1000 QR Images of Malicious and Benign QR codes 2025/benign_qr_images_500"
#malicious_folder = "/content/1000 QR Images of Malicious and Benign QR codes 2025/malicious_qr_images_500"

image_directory = "/content/1000 QR Images of Malicious and Benign QR codes 2025"

image_transforms = transforms.Compose([
      transforms.Resize((330, 330)), # Resize images to a consistent size
      transforms.ToTensor(),         # Convert PIL Image or NumPy array to PyTorch Tensor
  ])

image_dataset = datasets.ImageFolder(root=image_directory, transform=image_transforms)
image_trainset, image_testset = train_test_split(image_dataset, test_size=0.2, random_state=42)

batch_size = 32
image_loader = DataLoader(image_trainset, batch_size=batch_size, shuffle=True)

test_loader = DataLoader(image_testset, batch_size=batch_size, shuffle=False)

In [ ]:
for batch_idx, (images, labels) in enumerate(image_loader):
      # 'images' is a PyTorch tensor containing a batch of images
      # 'labels' is a PyTorch tensor containing the corresponding labels (if applicable)
      print(f"Batch {batch_idx}: Images tensor shape - {images.shape}")

Batch 0: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 1: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 2: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 3: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 4: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 5: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 6: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 7: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 8: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 9: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 10: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 11: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 12: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 13: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 14: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 15: Images tensor shape - torch.Size([32, 3, 330, 330])
Batch 16: Images t

In [ ]:
class Classification_Conv_NN(nn.Module):
  def __init__(self):
    super(Classification_Conv_NN, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.fc1 = nn.Linear(64 * 82 * 82, 128)
    self.fc2 = nn.Linear(128, 1)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 64 * 82 * 82)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [ ]:
model = Classification_Conv_NN()

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model

Classification_Conv_NN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=430336, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Classification_Conv_NN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=430336, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)

In [ ]:
# Training

# Defining variables
epochs = 10
losses = []

# Training loop
for epoch in range(epochs):
    for batch_idx, (images, labels) in enumerate(image_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

Epoch 1/10, Loss: 0.4358364939689636
Epoch 2/10, Loss: 0.5641852021217346
Epoch 3/10, Loss: 0.29927098751068115
Epoch 4/10, Loss: 0.38086608052253723
Epoch 5/10, Loss: 0.36125990748405457
Epoch 6/10, Loss: 0.19887368381023407
Epoch 7/10, Loss: 0.07671874761581421
Epoch 8/10, Loss: 0.01778755895793438
Epoch 9/10, Loss: 0.13163986802101135
Epoch 10/10, Loss: 0.036784835159778595


In [ ]:
# Evaluation

model.eval() # Set the model to evaluation mode

with torch.no_grad(): # Disable gradient calculation
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predicted = (outputs.squeeze() > 0.5).long() # Apply threshold and convert to long

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy on the test set: {accuracy:.2f}%")

Accuracy on the test set: 88.50%


In [ ]:
# Save the model
torch.save(model.state_dict(), 'classification_conv_nn.pth')
print("Model saved successfully!")

Model saved successfully!
